In [4]:
import pickle

In [5]:
filename = "../CodeSearchNet/big_csn_data/CodeSearchNet_Java_Search_Preliminary.pkl"
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [48]:
len(new_dict)

1569889

In [46]:
i = -1 #+(100000)*15
print(new_dict[i]['function'])

private static String interpretFacet(final Facet facet) {
        if (facet == null || facet.isNoop()) {
            return "";
        }
        if (facet instanceof ImperativeFacet) {
            ImperativeFacet imperativeFacet = (ImperativeFacet) facet;
            return imperativeFacet.getMethods().get(0).getName();
        } 
        final String name = facet.getClass().getSimpleName();
        if (ignore(name)) {
            return "";
        } 
        final String abbr = StringExtensions.toAbbreviation(name);
        return abbr.length()>0 ? abbr : name;
    }


In [47]:
len(new_dict[i]['function_tokens'])

110

## Analyzing Compression

In [1]:
import json
import copy

In [2]:
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

In [3]:
def getJsonDataLineByLine(JsonFile):
    data = []
    i = 0
    with open(JsonFile) as f:
        for line in f:
            i += 1
            line = line.strip()
            if line[0] == '[':
                line = line[1:]
            elif line[-1] == ']':
                line = line[:-1]
            if line[-1] == ',':
                line = line[:-1]
            try:
                data.append(json.loads(line))
                if i%10000 == 0: print(i, "success")
            except:
                data.append('''{"classBodyDeclaration": ""}''')
                print(i, "failure")
                print(line)
    return data

In [4]:
def getJsonDataSafe(JsonFile):
    try:
        return getJsonData(JsonFile)
    except Exception as e:
        print(e)
        print("Loading JSON failed. Attempting safe method.")
        return getJsonDataLineByLine(JsonFile)

In [75]:
def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

# Visualize Tree

In [81]:
def travarse(tree):
    if 'type' in tree:
        print(tree['text'], end=" ")
        if tree['text'] in [';', '{', '}']:
            print()
        return
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            travarse(child)

In [82]:
def tree2code(tree, indentation):
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        return code, indentation
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        code = ""
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
        return code, indentation

In [83]:
def printCode(tree):
    code, _ = tree2code(tree, 0)
    print(code)
    print("Count:", len(code.split()))

In [9]:
ls ../csn_big_only_trees/

csn_big_10_compressed.json    csn_big_2_compressed.json
csn_big_10.json               csn_big_2.json
csn_big_11u_compressed.json   csn_big_3_compressed.json
csn_big_11u.json              csn_big_3.json
csn_big_12_compressed.json    csn_big_4u_compressed.json
csn_big_12.json               csn_big_4u.json
csn_big_13uu_compressed.json  csn_big_5_compressed.json
csn_big_13uu.json             csn_big_5.json
csn_big_14_compressed.json    csn_big_6_compressed.json
csn_big_14.json               csn_big_6.json
csn_big_15_compressed.json    csn_big_7u_compressed.json
csn_big_15.json               csn_big_7u.json
csn_big_16_compressed.json    csn_big_8_compressed.json
csn_big_16_d.json             csn_big_8.json
csn_big_1u_compressed.json    csn_big_9u_compressed.json
csn_big_1u.json               csn_big_9u.json


In [55]:
uncompressed_data = getJsonData('../csn_big_only_trees/csn_big_1.json')

In [79]:
compressed_data = getJsonData('../CodeSearchNet/train_test_valid_data_10k/java_train_1u_10k_compressed.json')

In [84]:
bef_comp = 0
aft_comp = 0
for i in range(len(compressed_data)):
    code, _ = tree2code(uncompressed_data[i], 0)
    num_tokens = len(code.split())
    compressed_tokens = len(compressed_data[i]['code'].split())
    bef_comp += num_tokens
    aft_comp += compressed_tokens
    if not compressed_tokens:
        aft_comp += num_tokens

print(1-aft_comp/bef_comp)

0.1788277646998343


In [37]:
i = 205

In [62]:
code, _ = tree2code(uncompressed_data[i], 0)
num_tokens = len(code.split())
print(code)
print(num_tokens)
print("================================")
print(compressed_data[i]['code'])
compressed_tokens = len(compressed_data[i]['code'].split())
print(compressed_tokens)
print("Compression Ratio:", compressed_tokens/num_tokens)
i+=1

private List < ConfigurationPlaceholderResolver > loadAndOrderPlaceholderResolvers ( ) { 
	final List < ConfigurationPlaceholderResolver > configurationPlaceholderResolvers = new ArrayList < ConfigurationPlaceholderResolver > ( serviceLoaderInstance . get ( ) . all ( ConfigurationPlaceholderResolver . class ) ) ; 
	Collections . sort ( configurationPlaceholderResolvers , new Comparator < ConfigurationPlaceholderResolver > ( ) { 
		public int compare ( ConfigurationPlaceholderResolver firstResolver , ConfigurationPlaceholderResolver secondResolver ) { 
			Integer a = firstResolver . precedence ( ) ; 
			Integer b = secondResolver . precedence ( ) ; 
			return b . compareTo ( a ) ; 
			} 
		} 
	) ; 
	return configurationPlaceholderResolvers ; 
	} 

95
private List < ConfigurationPlaceholderResolver > loadAndOrderPlaceholderResolvers $F2 { 
	final List < ConfigurationPlaceholderResolver > configurationPlaceholderResolvers = new ArrayList < ConfigurationPlaceholderResolver > ( $F144 ( serv

0.2104572106016639


In [ ]:
0.2104572106016639

## How many of the compressed tokens were present in the top 10k train data?

In [3]:
abstract_compressed_tokens = getJsonData('../CodeSearchNet/abstract_code_dict_1.5m.json')

In [6]:
abstract_compressed_list = sorted(abstract_compressed_tokens, key = lambda x:abstract_compressed_tokens[x]['count'], reverse=True)

In [8]:
abstract_compressed_tokens[abstract_compressed_list[0]]

{'count': 782542, 'compressed_name': '$F2', 'num_params': 0}

In [9]:
most_freq_25k = [abstract_compressed_tokens[abstract_compressed_list[i]]['compressed_name'] for i in range(25000)]

In [52]:
most_freq_10k = [abstract_compressed_tokens[abstract_compressed_list[i]]['compressed_name'] for i in range(10000)]

In [30]:
most_freq_25k[:1000]

['$F2',
 '$F1',
 '$F268',
 '$F112',
 '$F332',
 '$F62338',
 '$F1372',
 '$F641',
 '$F198',
 '$F6',
 '$F3413',
 '$F527',
 '$F724',
 '$F528',
 '$F50',
 '$F45',
 '$F1217',
 '$F507',
 '$F254',
 '$F797',
 '$F320',
 '$F1218',
 '$F3745',
 '$F213',
 '$F209',
 '$F1975',
 '$F1203',
 '$F564',
 '$F232',
 '$F356',
 '$F1286',
 '$F384',
 '$F269',
 '$F21',
 '$F1987',
 '$F199',
 '$F60794',
 '$F2443',
 '$F5033',
 '$F2612',
 '$F144',
 '$F5755',
 '$F2200',
 '$F95',
 '$F4356',
 '$F23605',
 '$F1645',
 '$F3956',
 '$F51',
 '$F17184',
 '$F712',
 '$F298',
 '$F21776',
 '$F304',
 '$F1292',
 '$F11311',
 '$F14426',
 '$F2724',
 '$F762',
 '$F1211',
 '$F2725',
 '$F1108',
 '$F1109',
 '$F11309',
 '$F96',
 '$F255',
 '$F1668',
 '$F6210',
 '$F10937',
 '$F2377',
 '$F2015',
 '$F6610',
 '$F5605',
 '$F867',
 '$F25224',
 '$F52',
 '$F2074',
 '$F511',
 '$F53',
 '$F29197',
 '$F5249',
 '$F1212',
 '$F890',
 '$F2674',
 '$F3506',
 '$F197020',
 '$F1357',
 '$F1169',
 '$F54',
 '$F385',
 '$F2233',
 '$F560',
 '$F4241',
 '$F4357',
 '$F24429',

In [43]:
#!mkdir ../CodeSearchNet/25k
#!mv ../CodeSearchNet/train-test-valid_25kf.zip ../CodeSearchNet/25k/
#!unzip ../CodeSearchNet/25k/train-test-valid_25kf.zip
!ls ../CodeSearchNet/train_test_valid_data_10k/

java_test_0_10k_compressed.json     java_train_2u_10k_compressed.json
java_train_0u_10k_compressed.json   java_train_3u_10k_compressed.json
java_train_10_10k_compressed.json   java_train_4u_10k_compressed.json
java_train_11u_10k_compressed.json  java_train_5u_10k_compressed.json
java_train_12_10k_compressed.json   java_train_6u_10k_compressed.json
java_train_13_10k_compressed.json   java_train_7u_10k_compressed.json
java_train_14_10k_compressed.json   java_train_8_10k_compressed.json
java_train_15u_10k_compressed.json  java_train_9_10k_compressed.json
java_train_1u_10k_compressed.json   java_valid_0_10k_compressed.json


In [46]:
import glob
dirs = glob.glob('../CodeSearchNet/train_test_valid_data_10k/*')
dirs

['../CodeSearchNet/train_test_valid_data_10k/java_valid_0_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_14_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_2u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_8_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_5u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_test_0_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_6u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_9_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_13_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_15u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_1u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_0u_10k_compressed.json',
 '../CodeSearchNet/train_test_va

In [47]:
freq_dict = {}['../CodeSearchNet/train_test_valid_data_10k/java_valid_0_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_14_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_2u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_8_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_5u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_test_0_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_6u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_9_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_13_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_15u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_1u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_0u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_3u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_12_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_7u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_11u_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_10_10k_compressed.json',
 '../CodeSearchNet/train_test_valid_data_10k/java_train_4u_10k_compressed.json']
freq_dict = {}
for path in dirs:
    dataset = getJsonData(path)
    for data in dataset:
        code = data['code']
        code_tokens = code.split()
        for token in code_tokens:
            if token not in freq_dict:
                freq_dict[token] = 0
            freq_dict[token] += 1

In [48]:
len(freq_dict)

1160247

In [ ]:
# 25k compressed total tokens: 1024352
# 10k compressed total tokens: 1160247

In [49]:
data_freq_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)

In [50]:
top_data_freq_list[:1000]

[')',
 '(',
 ',',
 '.',
 ';',
 '{',
 '}',
 '=',
 'if',
 'public',
 'new',
 'return',
 'String',
 '>',
 '<',
 'this',
 'final',
 'STRING00',
 'INT00',
 'else',
 ':',
 'i',
 '$F2',
 '+',
 'void',
 'null',
 'int',
 'e',
 'for',
 'static',
 '[',
 ']',
 'throws',
 'result',
 'try',
 '==',
 'value',
 '$F1',
 '&&',
 'catch',
 'private',
 'throw',
 'boolean',
 '$F268',
 '!=',
 'name',
 '!',
 '-',
 '||',
 'case',
 'key',
 '$F112',
 '?',
 '*',
 'List',
 'IOException',
 'type',
 'tc',
 'request',
 'protocolMarshaller',
 'class',
 'add',
 'c',
 'Object',
 '$F332',
 'true',
 'marshall',
 'protected',
 'index',
 'double',
 'response',
 'long',
 'instanceof',
 'while',
 'get',
 'input',
 'state',
 'context',
 'sb',
 '++',
 'false',
 'System',
 'length',
 'Exception',
 'path',
 'x',
 'data',
 's',
 'j',
 't',
 'put',
 'entry',
 'logger',
 'log',
 '$F1372',
 'append',
 'Integer',
 'node',
 'b',
 'n',
 'out',
 'id',
 '$F1218',
 'size',
 'p',
 'T',
 'equals',
 '$F1987',
 'LOG',
 'a',
 '+=',
 'ex',
 'buff

In [61]:
top_data_freq_list = data_freq_list[:10000]

In [53]:
common_tokens = list(set(top_data_freq_list) & set(most_freq_10k))

In [54]:
len(common_tokens)

6889

In [85]:
len(top_data_freq_list)

10000

In [87]:
list_10k_10k = []

In [88]:
for token in top_data_freq_list:
    list_10k_10k.append({'token':token, 'count':freq_dict[token]})

In [89]:
additional_compressed_tokens = list(set(most_freq_10k) - set(top_data_freq_list))
len(additional_compressed_tokens)

6804

In [90]:
for token in additional_compressed_tokens:
    if token in freq_dict:
        list_10k_10k.append({'token':token, 'count':freq_dict[token]})

In [91]:
len(list_10k_10k)

15104

In [92]:
dump_dict(list_10k_10k, "list_10k_10k.json")